In [23]:
# CRA Project #1 Wholesale PD Model 
# Chris Yu Yan
import pandas as pd
import numpy as np
import datetime as dt
from sklearn import metrics
import statsmodels.api as sm
import itertools

bankdata = pd.read_pickle('bank financials.pickle')
bankdata.head()

,IDRSSD,name,mutual,trust,rssdhcr,date,TII,nonII,nonIE,staff,...,cash,securities,loans,deposits,allowance,equity,TCOs,tot_recoveries,offices,ones
0,991340,1st Source Bank,Stock,1,1199602.0,2001-12-31,236773,72464,112798,1135,...,130135.0,620621.0,2306365.0,5767558.0,57624.0,320511.0,3598.0,188,66,1.0
1,282208,Abington Savings Bank,Stock,0,2521741.0,2001-12-31,50492,12328,25420,277,...,21689.0,277627.0,380837.0,999000.0,5482.0,51238.0,73.0,37,14,1.0
2,5210,Adams County National Bank,Stock,1,1117464.0,2001-12-31,39161,3298,13975,168,...,21925.0,175017.0,358856.0,1019216.0,3723.0,61521.0,167.0,52,18,1.0
3,774730,Admiralty Bank,Stock,0,2142548.0,2001-12-31,30363,1167,11839,117,...,16275.0,17955.0,390457.0,899350.0,4844.0,42775.0,9.0,0,10,1.0
4,2079493,Advance Bank,Stock,1,1137060.0,2001-12-31,41168,3964,19838,246,...,12818.0,176529.0,403514.0,967834.0,3378.0,48741.0,3853.0,39,15,1.0


In [24]:
# Step 1 - Cleaning the Financial Data
bankdata['total_rev'] = bankdata['nonII']+bankdata['TII']
bankdata['expense'] = bankdata['nonIE']/bankdata['total_rev']
bankdata['loan_to_deposit'] = bankdata['loans']/bankdata['deposits']
bankdata['acl_to_loans'] = bankdata['allowance']/bankdata['loans']

# Data Normalization
variables = ['TII', 'nonIE', 'nonII', 'deposits', 'staff', 'cash', 'securities', 'equity', 'TCOs', 'tot_recoveries', 'allowance']
for var in variables:
    bankdata[var] = bankdata[var]/bankdata['assets']


In [25]:
# Step 2 - Adding in the Default Data
defaults = pd.read_pickle('default_data.pickle')

merged_data = pd.merge(pd.DataFrame(bankdata), pd.DataFrame(defaults)[['IDRSSD', 'default', 'dflt_date']], on = 'IDRSSD', how = 'left')
merged_data['default'].fillna(0, inplace=True)

merged_data['date'] = pd.to_datetime(merged_data['date'])
merged_data['dflt_date'] = pd.to_datetime(merged_data['dflt_date'])
merged_data['days_diff'] = (merged_data['dflt_date'] - merged_data['date']).dt.days
merged_data.loc[(merged_data['default'] == 1) & ((merged_data['days_diff'] < 183) | (merged_data['days_diff'] > 549)), 'default'] = 0
merged_data.loc[merged_data['default'] == 0, 'dflt_date'] = dt.date(2999, 12, 31)
merged_data.drop(columns=['days_diff'], inplace=True)
merged_data.head()

C:\Users\jwhw0\AppData\Local\Temp\ipykernel_28216\3305419862.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2999-12-31' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  merged_data.loc[merged_data['default'] == 0, 'dflt_date'] = dt.date(2999, 12, 31)


,IDRSSD,name,mutual,trust,rssdhcr,date,TII,nonII,nonIE,staff,...,TCOs,tot_recoveries,offices,ones,total_rev,expense,loan_to_deposit,acl_to_loans,default,dflt_date
0,991340,1st Source Bank,Stock,1,1199602.0,2001-12-31,0.067905,0.020782,0.032350,0.000326,...,0.001032,0.000054,66,1.0,309237,0.364762,0.399886,0.024985,0.0,2999-12-31
1,282208,Abington Savings Bank,Stock,0,2521741.0,2001-12-31,0.065565,0.016008,0.033008,0.000360,...,0.000095,0.000048,14,1.0,62820,0.404648,0.381218,0.014395,0.0,2999-12-31
2,5210,Adams County National Bank,Stock,1,1117464.0,2001-12-31,0.062404,0.005255,0.022269,0.000268,...,0.000266,0.000083,18,1.0,42459,0.329141,0.352090,0.010375,0.0,2999-12-31
3,774730,Admiralty Bank,Stock,0,2142548.0,2001-12-31,0.061024,0.002345,0.023794,0.000235,...,0.000018,0.000000,10,1.0,31530,0.375484,0.434155,0.012406,0.0,2999-12-31
4,2079493,Advance Bank,Stock,1,1137060.0,2001-12-31,0.067077,0.006459,0.032323,0.000401,...,0.006278,0.000064,15,1.0,45132,0.439555,0.416925,0.008371,0.0,2999-12-31


In [26]:
# Step 3 - Single Factor Analysis
# Correlation Analysis
new_merged_data = merged_data.copy()
new_merged_data = new_merged_data.drop(columns=['name', 'mutual', 'date', 'dflt_date'])

correlations = new_merged_data.corr()
correlation_with_default = correlations['default']

print("Correlations with 'default':")
print(correlation_with_default)

# AUC Analysis
variables = ['TII', 'nonIE', 'nonII', 'deposits', 'staff', 'cash', 'securities', 'equity', 'TCOs', 'tot_recoveries', 'allowance']

auc_scores = {}
for var in variables:
    auc = metrics.roc_auc_score(new_merged_data['default'], new_merged_data[var])
    auc_scores[var] = auc

print("\nAUC Scores for Each Variable:")
for var, auc in auc_scores.items():
    print(f"{var}: {auc}")


Correlations with 'default':
IDRSSD             0.020104
trust             -0.027670
rssdhcr            0.015586
TII                0.056560
nonII             -0.020431
nonIE              0.014559
staff             -0.021437
assets             0.000004
cash              -0.004580
securities        -0.031886
loans             -0.000376
deposits           0.004535
allowance          0.149012
equity            -0.101917
TCOs               0.235382
tot_recoveries     0.004476
offices           -0.002586
ones                    NaN
total_rev         -0.001192
expense            0.012007
loan_to_deposit   -0.000465
acl_to_loans       0.139371
default            1.000000
Name: default, dtype: float64

AUC Scores for Each Variable:
TII: 0.7569033503863329
nonIE: 0.5860938384415598
nonII: 0.28512509445050793
deposits: 0.5618214120317083
staff: 0.4102192064261553
cash: 0.44701529624283337
securities: 0.3729124406552902
equity: 0.11970975595884528
TCOs: 0.8875019973404904
tot_recoveries: 0.499802

In [27]:
# Step 4 - Multi-factor Analysis
factors = ['expense', 'nonIE', 'loan_to_deposit', 'acl_to_loans', 'TII', 'nonII', 'loans', 'deposits', 'allowance'] 
factor_combinations = list(itertools.combinations(factors, 4))

results = []
for combination in factor_combinations:
    x = new_merged_data[list(combination)].copy()

    if not np.isfinite(x).all().all():
        x.replace([np.inf, -np.inf], np.nan, inplace=True)
        x.dropna(inplace=True)

    correlation_matrix = x.corr().abs()
    high_corr = (np.triu(correlation_matrix.values, 1) > 0.5).any()
    
    if not high_corr:
        x = sm.add_constant(x)
        y = new_merged_data['default'].loc[x.index]
        
        try:
            model = sm.Logit(y, x).fit(disp=0)
            predicted_probs = model.predict(x)
            auc = metrics.roc_auc_score(y, predicted_probs)
            results.append((combination, model, auc))
        except Exception as e:
            print(f"Error fitting model for combination {combination}: {e}")
            continue
        
results = sorted(results, key=lambda x: x[2], reverse=True)

# Output the top 3 models based on AUC and Pseudo-R-squared
print("Top 3 models based on AUC and Pseudo-R-squared:")
for idx, (combination, model, auc) in enumerate(results[:3], start=1):
    pseudo_r_squared = model.prsquared
    print(f"Model {idx}: Factors = {combination}, AUC = {auc}, Pseudo-R-squared = {pseudo_r_squared}")

# Based on the output, the AUC scores and Psuedo-R-squared are not significantly different. 
# Therefore, we will still proceed with the model with the highest scores.

# Print the summary of the best model (the one with the highest AUC)
best_combination, best_model, best_auc = results[0]
print(best_model.summary())


Top 3 models based on AUC and Pseudo-R-squared:
Model 1: Factors = ('expense', 'acl_to_loans', 'TII', 'nonII'), AUC = 0.8742017408684302, Pseudo-R-squared = 0.1389420893091019
Model 2: Factors = ('loan_to_deposit', 'acl_to_loans', 'TII', 'nonII'), AUC = 0.8727887835482652, Pseudo-R-squared = 0.1379840873173671
Model 3: Factors = ('acl_to_loans', 'TII', 'nonII', 'loans'), AUC = 0.8721779948597258, Pseudo-R-squared = 0.13796930720633593
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                23946
Model:                          Logit   Df Residuals:                    23941
Method:                           MLE   Df Model:                            4
Date:                Mon, 23 Sep 2024   Pseudo R-squ.:                  0.1389
Time:                        22:23:52   Log-Likelihood:                -668.46
converged:                       True   LL-Null:                       -776.32
Covarian

Model 1 might have the highest AUC (minimal difference) but shares many factors with Model 2. Therefore, Model 2 is not significantly different both quantitatively and qualitatively. Model 3 adds 'loans' as a factor instead of 'expense' or 'loan_to_deposit', making it qualitatively different. The AUC is still close to the top model, so it might not be significantly different quantitatively, but the difference in factors shows qualitative variation. Overall, the performance of all three top models are very close to each other in terms of strong predictive ability. 

In [28]:
# Step 5 - Predicting PD for Silicon Valley Bank and Signature Bank

silicon_valley_id = defaults[defaults['name'] == 'Silicon Valley Bank']['IDRSSD'].values[0]
signature_bank_id = defaults[defaults['name'] == 'Signature Bank']['IDRSSD'].values[0]

silicon_valley_data = new_merged_data[new_merged_data['IDRSSD'] == silicon_valley_id]
signature_bank_data = new_merged_data[new_merged_data['IDRSSD'] == signature_bank_id]

X_silicon_valley = silicon_valley_data[list(best_combination)]
X_silicon_valley = sm.add_constant(X_silicon_valley)
predicted_prob_silicon_valley = best_model.predict(X_silicon_valley)

X_signature = signature_bank_data[list(best_combination)]
X_signature = sm.add_constant(X_signature)
predicted_prob_signature = best_model.predict(X_signature)

print(f"Predicted probability of default for Silicon Valley Bank: {predicted_prob_silicon_valley.values}")
print(f"Predicted probability of default for Signature Bank: {predicted_prob_signature.values}")

actual_default_silicon_valley = silicon_valley_data['default'].values
actual_default_signature = signature_bank_data['default'].values

print(f"Actual default for Silicon Valley Bank: {actual_default_silicon_valley}")
print(f"Actual default for Signature Bank: {actual_default_signature}")


Predicted probability of default for Silicon Valley Bank: [0.00403906 0.00281773 0.00275168 0.00418642 0.0031149  0.00271634
 0.00232488 0.00391169 0.00467382 0.0048549  0.00454874 0.00418395
 0.00454653 0.00443681 0.00460508 0.00396953 0.00279167 0.00220095
 0.00212029 0.00324607 0.00316286]
Predicted probability of default for Signature Bank: [0.00548235 0.00521997 0.00413559 0.00445032 0.00426948 0.00452349
 0.00459984 0.00520133 0.00403529]
Actual default for Silicon Valley Bank: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Actual default for Signature Bank: [0. 0. 0. 0. 0. 0. 0. 0. 1.]


Based on the results, this model did not capture the defaults of Silicon Valley Bank and Signature Bank well.
This could be due to many different reasons including liquidity status, macroeconomic environment etc.
Therefore, other factors like interest rate levels, liquidity ratios, ROA, and ROE could have contributed to improve the model and achieve a better result.